In [1]:
import keras

Using TensorFlow backend.


In [2]:
!pip install np_utils

Data preprocessing

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
!pip install kaggle
!kaggle competitions download -c petfinder-adoption-prediction

/bin/sh: 1: kaggle: not found


In [4]:
data = pd.read_csv('Churn_Modelling.csv')
X = data.iloc[:, 3:13]
y = data.iloc[:, -1]

In [15]:
X_train.columns

Index(['CreditScore', 'Gender', 'Age', 'Balance', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Geography_Germany',
       'Geography_Spain', 'num_prods_2', 'num_prods_3', 'num_prods_4',
       'tenure_1', 'tenure_2', 'tenure_3', 'tenure_4', 'tenure_5', 'tenure_6',
       'tenure_7', 'tenure_8', 'tenure_9', 'tenure_10'],
      dtype='object')

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
"""def encode_independent(ind):
    le = LabelEncoder()
    
    X.iloc[:, ind] = le.fit_transform(X.iloc[:, ind])"""

le = LabelEncoder()    
X['Gender'] = le.fit_transform(X['Gender'])

In [6]:
X_geog = pd.get_dummies(X['Geography'], prefix = 'Geography', drop_first = True)
X = pd.concat([X, X_geog], axis = 1)

X.drop(['Geography'], axis = 1, inplace = True)

In [7]:
X_numprods = pd.get_dummies(X['NumOfProducts'], prefix = 'num_prods', drop_first = True)
X = pd.concat([X, X_numprods], axis = 1)
X.drop(['NumOfProducts'], axis = 1, inplace = True)

In [8]:
X_tenure = pd.get_dummies(X['Tenure'], prefix = 'tenure', drop_first = True)
X = pd.concat([X, X_tenure], axis = 1)
X.drop(['Tenure'], axis = 1, inplace = True)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [10]:
#feature scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()


In [11]:
X_train[['CreditScore', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_train[['CreditScore', 'Balance', 'EstimatedSalary']])
X_val[['CreditScore', 'Balance', 'EstimatedSalary']] = sc.transform(X_val[['CreditScore', 'Balance', 'EstimatedSalary']])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [12]:
y_train_temp = y_train.copy()
y_val_temp = y_val.copy()
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

Making the ANN

In [40]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [41]:
#Initializing the Keras model
classifier = Sequential()

In [42]:
#adding dense layers
classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu', input_dim = 22))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=22, units=12, kernel_initializer="uniform", activation="relu")`
  


In [43]:

classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12, kernel_initializer="uniform", activation="relu")`
  


In [44]:
classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12, kernel_initializer="uniform", activation="relu")`
  """Entry point for launching an IPython kernel.


In [45]:
classifier.add(Dense(output_dim = 2, init = 'uniform', activation = 'softmax'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2, kernel_initializer="uniform", activation="softmax")`
  """Entry point for launching an IPython kernel.


In [46]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1, mode='auto')]

In [47]:
#compile the ANN
from keras.optimizers import Adam
adam = Adam(lr = 1e-2)
classifier.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#training 
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight('balanced',
                                                 np.unique(y_train_temp),
                                                 y_train_temp)
classifier.fit(X_train, y_train, batch_size = 20, nb_epoch = 100, callbacks = callbacks, validation_split = 0.2, class_weight = class_weights)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 2s 373us/step - loss: 0.4702 - acc: 0.7940 - val_loss: 0.4340 - val_acc: 0.7769
Epoch 2/100
6400/6400 [==============================] - 1s 213us/step - loss: 0.4107 - acc: 0.8050 - val_loss: 0.4396 - val_acc: 0.8081
Epoch 3/100
6400/6400 [==============================] - 1s 217us/step - loss: 0.4038 - acc: 0.8289 - val_loss: 0.4169 - val_acc: 0.8206
Epoch 4/100
6400/6400 [==============================] - 1s 218us/step - loss: 0.3978 - acc: 0.8334 - val_loss: 0.4425 - val_acc: 0.8063
Epoch 5/100
6400/6400 [==============================] - 1s 219us/step - loss: 0.3914 - acc: 0.8356 - val_loss: 0.4260 - val_acc: 0.8113
Epoch 6/100
6400/6400 [==============================] - 1s 218us/step - loss: 0.3903 - acc: 0.8387 - val_loss: 0.4076 - val_acc: 0.8212
Epoch 7/100
6400/6400 [==============================] - 1s 227us/step - loss: 0.3857 - acc: 0.8395 - val_loss: 0.4048 - val_acc: 

Now that model has been trained to 85% accuracy , start with test/validation set

In [ ]:
classifier.evaluate(X_val, y_val)

In [25]:
y_pred = classifier.predict_classes(X_val)

In [27]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val_temp, y_pred)

In [28]:
cm

array([[1604,   23],
       [ 256,  117]])

In [ ]:
#false negatives are only 23 , so only 23 customers might leave without it being predicted?

In [39]:
y_pred_prob = classifier.predict(X_val)
y_pred1 = (y_pred_prob[:, 1] > 0.5)
cm1 = confusion_matrix(y_val_temp, y_pred1)
cm1

#IMP: CAN SET CONFIDENCE THIS WAY

array([[1604,   23],
       [ 256,  117]])

In [ ]:
#same results